In [1]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
from IPython.display import display

In [2]:
URI = "dbbikes.cfjfzkae45jy.eu-west-1.rds.amazonaws.com"
PORT="3306"
DB="dbbikes"
USER="admin"
PASSWORD = "mypassword"


In [3]:
engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [4]:
sql = """
CREATE DATABASE IF NOT EXISTS dbbikes;
"""

engine.execute(sql)

2023-02-27 19:51:39,099 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-02-27 19:51:39,101 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:51:39,137 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-02-27 19:51:39,138 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:51:39,157 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-02-27 19:51:39,158 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:51:39,200 INFO sqlalchemy.engine.Engine 
CREATE DATABASE IF NOT EXISTS dbbikes;

2023-02-27 19:51:39,201 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:51:39,230 INFO sqlalchemy.engine.Engine COMMIT


## Checking if the connection has been established by running a simple SQL code.

In [5]:
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-02-27 19:51:41,285 INFO sqlalchemy.engine.Engine SHOW VARIABLES;
2023-02-27 19:51:41,286 INFO sqlalchemy.engine.Engine [raw sql] ()
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('back_log', '61')
('basedir', '/rdsdbbin/mysql-8.0.28.R4/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_direct_non_transactional_updates', 'OFF')
('binlog_encryption', 'OFF')
('binlog_error_action', 'ABORT_SERVER')
('binlog_expire_logs_seconds', '2592000')

In [6]:
sql = """
CREATE TABLE IF NOT EXISTS station (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contract_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)
"""

try:
    res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-27 19:51:43,796 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS station
2023-02-27 19:51:43,797 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:51:43,873 INFO sqlalchemy.engine.Engine COMMIT
2023-02-27 19:51:43,911 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS station (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contract_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)

2023-02-27 19:51:43,912 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:51:43,996 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


A 'station' table was successfully created. Note that 'banking' and 'bonus' columns should really take boolean values, however, SQL doesn't have boolean data type, so I went with INT, where 0 is 'no' and 1 is 'yes'. In the scraped JSON, these values are expressed as boolean.

In [7]:
sql = """
CREATE TABLE IF NOT EXISTS availability(
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)
"""

try:
    res = engine.execute("DROP TABLE IF EXISTS availability")
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-27 19:51:45,407 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS availability
2023-02-27 19:51:45,408 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:51:45,452 INFO sqlalchemy.engine.Engine COMMIT
2023-02-27 19:51:45,496 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS availability(
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)

2023-02-27 19:51:45,497 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:51:45,542 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [8]:
# #!/usr/bin/env python

# APIKEY = "a471198f1d4a279171da8f17892b64eb12c32f33"
# NAME = "Dublin"
# STATIONS = "https://api.jcdecaux.com/vls/v1/stations"

# # api_key = 'a471198f1d4a279171da8f17892b64eb12c32f33'
# # contract_name = 'dublin'
# # api_query = 'https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey=a471198f1d4a279171da8f17892b64eb12c32f33'


# # def write_to_file(text):
# #     with open("data/bikes_{}".format(now).replace(" ", "_"), "w") as f:
# #         f.write(r.text)
        
# # def write_to_db(text): #?????
 
    
# def main():
#     while True:
#         try:
#             now = datetime.datetime.now()
#             r = requests.get(STATIONS, params= {"apiKey": APIKEY, "contract": NAME})
#             print(r, now)
            
# #             write_to_file(r.text)
# #             write_to_db(r.text)
#             time.sleep(300)
#         except:
#             print(traceback.format_exc())
# #             if engine is None:
# #                 pass #?????
#     return
# main()
    

#  !IMPORTANT! 

Right now, the main( ) function only runs once. When it is pushed to EC2, it needs to be changed a bit, so that it runs non-stop.

In [9]:
api_key = 'a471198f1d4a279171da8f17892b64eb12c32f33'
contract_name = 'dublin'
api_query = 'https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey=a471198f1d4a279171da8f17892b64eb12c32f33'



def main():
    #while True:
    global r
    try:
        r = requests.get('https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey=a471198f1d4a279171da8f17892b64eb12c32f33')
        # Keep the scraped data as a variable in the code without saving it anywhere
        # with open('json_data.json', 'w') as outfile:
        #     json.dump(data, outfile)
        print(json.loads(r.text))
        
        #time.sleep(5*60) #Scrape every 5 minutes
    except:
        print("Error. Something went wrong.") 
    #return
main()

[{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 4, 'available_bikes': 26, 'status': 'OPEN', 'last_update': 1677527192000}, {'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 17, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1677527305000}, {'number': 54, 'contract_name': 'dublin', 'name': 'CLONMEL STREET', 'address': 'Clonmel Street', 'position': {'lat': 53.336021, 'lng': -6.26298}, 'banking': False, 'bonus': False, 'bike_stands': 33, 'available_bike_stands': 32, 'available_bikes': 1, 'status': 'OPEN', 'last_update': 1677527226000}, {'number': 108, 'contract_name': 'dublin', 'name': 'AVONDALE ROAD', '

In [10]:
def station_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get('address'),
                int(station.get('banking')),
                station.get('bike_stands'),
                int(station.get('bonus')),
                station.get('contract_name'),
                station.get('name'),
                station.get('number'),
                station.get('position').get('lat'),
                station.get('position').get('lng'),
                station.get('status'))
        engine.execute("insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
        break #Because of this break, the loop only inserts the first Station. Which is no.42.
    return

station_to_db(r.text)

<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 4, 'available_bikes': 26, 'status': 'OPEN', 'last_update': 1677527192000}
2023-02-27 19:51:49,024 INFO sqlalchemy.engine.Engine insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-02-27 19:51:49,026 INFO sqlalchemy.engine.Engine [raw sql] ('Smithfield North', 0, 30, 0, 'dublin', 'SMITHFIELD NORTH', 42, 53.349562, -6.278198, 'OPEN')
2023-02-27 19:51:49,055 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
metadata = sqla.MetaData()
station = sqla.Table("station", metadata,
                    sqla.Column('address', sqla.String(256), nullable=False),
                    sqla.Column('banking', sqla.Integer),
                    sqla.Column('bike_stand', sqla.Integer),
                    sqla.Column('bonus', sqla.Integer),
                    sqla.Column('contract_name', sqla.String(256)),
                    sqla.Column('name', sqla.String(256)),
                    sqla.Column('number', sqla.Integer),
                    sqla.Column('position_lat', sqla.REAL),
                    sqla.Column('position_lgt', sqla.REAL),
                    sqla.Column('status', sqla.String(256)))

availability = sqla.Table("availability", metadata,
                        sqla.Column('available_bikes', sqla.Integer),
                        sqla.Column('available_bike_stands', sqla.Integer),
                        sqla.Column('number', sqla.Integer),
                        sqla.Column('last_update', sqla.BigInteger))

try:
    station.drop(engine)
    availability.drop(engine)
except:
    pass

metadata.create_all(engine)

2023-02-27 19:51:50,866 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-27 19:51:50,867 INFO sqlalchemy.engine.Engine 
DROP TABLE station
2023-02-27 19:51:50,868 INFO sqlalchemy.engine.Engine [no key 0.00105s] ()
2023-02-27 19:51:50,907 INFO sqlalchemy.engine.Engine COMMIT
2023-02-27 19:51:50,953 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-27 19:51:50,955 INFO sqlalchemy.engine.Engine 
DROP TABLE availability
2023-02-27 19:51:50,956 INFO sqlalchemy.engine.Engine [no key 0.00104s] ()
2023-02-27 19:51:50,989 INFO sqlalchemy.engine.Engine COMMIT
2023-02-27 19:51:51,028 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-27 19:51:51,032 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-02-27 19:51:51,033 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ('dbbikes', 'station')
2023-02-27 19:51:51,051 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_sch

In [12]:
#Loading Tables that are already created


# metadata = sqla.MetaData(bind=engine)
# print(metadata)
# station=sqla.Table('station', metadata, autoload=True)
# print(station)


In [21]:
def stations_fix_keys(station):
    station['position_lat'] = station['position']['lat']
    station['position_lng'] = station['position']['lng']
    return station

stations = json.loads(r.text)

engine.execute(station.insert(), *map(stations_fix_keys, stations))


2023-02-27 19:52:41,800 INFO sqlalchemy.engine.Engine INSERT INTO station (address, banking, bonus, contract_name, name, number, position_lat, status) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
2023-02-27 19:52:41,802 INFO sqlalchemy.engine.Engine [cached since 40.7s ago] (('Smithfield North', False, False, 'dublin', 'SMITHFIELD NORTH', 42, 53.349562, 'OPEN'), ('Parnell Square North', False, False, 'dublin', 'PARNELL SQUARE NORTH', 30, 53.3537415547453, 'OPEN'), ('Clonmel Street', False, False, 'dublin', 'CLONMEL STREET', 54, 53.336021, 'OPEN'), ('Avondale Road', False, False, 'dublin', 'AVONDALE ROAD', 108, 53.359405, 'OPEN'), ('James Street East', False, False, 'dublin', 'JAMES STREET EAST', 20, 53.336597, 'OPEN'), ('Mount Street Lower', False, False, 'dublin', 'MOUNT STREET LOWER', 56, 53.33796, 'OPEN'), ('Christchurch Place', False, False, 'dublin', 'CHRISTCHURCH PLACE', 6, 53.343368, 'OPEN'), ('Grantham Street', False, False, 'dublin', 'GRANTHAM STREET', 18, 53.334123, 'OPEN')  ... d

In [14]:
def availability_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (int(station.get('number')),
                int(station.get('available_bikes')),
                int(station.get('available_bike_stands')),
                int(station.get('last_update')))
        
        engine.execute("insert into availability values(%s,%s,%s,%s)", vals)
    return

availability_to_db(r.text)

<class 'list'> 114
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 4, 'available_bikes': 26, 'status': 'OPEN', 'last_update': 1677527192000}
2023-02-27 19:52:03,911 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
2023-02-27 19:52:03,911 INFO sqlalchemy.engine.Engine [raw sql] (42, 26, 4, 1677527192000)
2023-02-27 19:52:03,940 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 17, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1677527305000}
2023-02-27 19:52:03,981 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
20

2023-02-27 19:52:04,794 INFO sqlalchemy.engine.Engine [raw sql] (98, 12, 28, 1677526946000)
2023-02-27 19:52:04,813 INFO sqlalchemy.engine.Engine COMMIT
{'number': 14, 'contract_name': 'dublin', 'name': 'FOWNES STREET UPPER', 'address': 'Fownes Street Upper', 'position': {'lat': 53.344603, 'lng': -6.263371}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 20, 'available_bikes': 10, 'status': 'OPEN', 'last_update': 1677527386000}
2023-02-27 19:52:04,858 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
2023-02-27 19:52:04,859 INFO sqlalchemy.engine.Engine [raw sql] (14, 10, 20, 1677527386000)
2023-02-27 19:52:04,878 INFO sqlalchemy.engine.Engine COMMIT
{'number': 1, 'contract_name': 'dublin', 'name': 'CLARENDON ROW', 'address': 'Clarendon Row', 'position': {'lat': 53.340927, 'lng': -6.262501}, 'banking': False, 'bonus': False, 'bike_stands': 31, 'available_bike_stands': 13, 'available_bikes': 18, 'status': 'OPEN', 'last_update': 167

2023-02-27 19:52:05,841 INFO sqlalchemy.engine.Engine [raw sql] (72, 1, 30, 1677527436000)
2023-02-27 19:52:05,860 INFO sqlalchemy.engine.Engine COMMIT
{'number': 63, 'contract_name': 'dublin', 'name': 'FENIAN STREET', 'address': 'Fenian Street', 'position': {'lat': 53.341428, 'lng': -6.24672}, 'banking': False, 'bonus': False, 'bike_stands': 35, 'available_bike_stands': 24, 'available_bikes': 11, 'status': 'OPEN', 'last_update': 1677527404000}
2023-02-27 19:52:05,903 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
2023-02-27 19:52:05,905 INFO sqlalchemy.engine.Engine [raw sql] (63, 11, 24, 1677527404000)
2023-02-27 19:52:05,921 INFO sqlalchemy.engine.Engine COMMIT
{'number': 113, 'contract_name': 'dublin', 'name': 'MERRION SQUARE SOUTH', 'address': 'Merrion Square South', 'position': {'lat': 53.338614, 'lng': -6.248606}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 31, 'available_bikes': 9, 'status': 'OPEN', 'last_update': 16

2023-02-27 19:52:06,818 INFO sqlalchemy.engine.Engine [raw sql] (49, 21, 19, 1677527328000)
2023-02-27 19:52:06,846 INFO sqlalchemy.engine.Engine COMMIT
{'number': 19, 'contract_name': 'dublin', 'name': 'HERBERT PLACE', 'address': 'Herbert Place', 'position': {'lat': 53.334432, 'lng': -6.245575}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 30, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1677527078000}
2023-02-27 19:52:06,894 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
2023-02-27 19:52:06,895 INFO sqlalchemy.engine.Engine [raw sql] (19, 0, 30, 1677527078000)
2023-02-27 19:52:06,915 INFO sqlalchemy.engine.Engine COMMIT
{'number': 7, 'contract_name': 'dublin', 'name': 'HIGH STREET', 'address': 'High Street', 'position': {'lat': 53.343565, 'lng': -6.275071}, 'banking': False, 'bonus': False, 'bike_stands': 29, 'available_bike_stands': 16, 'available_bikes': 13, 'status': 'OPEN', 'last_update': 1677527241000}
2023-0

2023-02-27 19:52:07,783 INFO sqlalchemy.engine.Engine [raw sql] (27, 6, 14, 1677527068000)
2023-02-27 19:52:07,801 INFO sqlalchemy.engine.Engine COMMIT
{'number': 16, 'contract_name': 'dublin', 'name': 'GEORGES QUAY', 'address': 'Georges Quay', 'position': {'lat': 53.347508, 'lng': -6.252192}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 11, 'available_bikes': 9, 'status': 'OPEN', 'last_update': 1677526971000}
2023-02-27 19:52:07,846 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
2023-02-27 19:52:07,847 INFO sqlalchemy.engine.Engine [raw sql] (16, 9, 11, 1677526971000)
2023-02-27 19:52:07,870 INFO sqlalchemy.engine.Engine COMMIT
{'number': 96, 'contract_name': 'dublin', 'name': 'KILMAINHAM LANE', 'address': 'Kilmainham Lane', 'position': {'lat': 53.341805, 'lng': -6.305085}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 27, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1677526918000}
2

2023-02-27 19:52:08,812 INFO sqlalchemy.engine.Engine [raw sql] (36, 1, 39, 1677527450000)
2023-02-27 19:52:08,828 INFO sqlalchemy.engine.Engine COMMIT
{'number': 93, 'contract_name': 'dublin', 'name': 'HEUSTON STATION (CENTRAL)', 'address': 'Heuston Station (Central)', 'position': {'lat': 53.346603, 'lng': -6.296924}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 14, 'available_bikes': 26, 'status': 'OPEN', 'last_update': 1677527052000}
2023-02-27 19:52:08,874 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
2023-02-27 19:52:08,876 INFO sqlalchemy.engine.Engine [raw sql] (93, 26, 14, 1677527052000)
2023-02-27 19:52:08,891 INFO sqlalchemy.engine.Engine COMMIT
{'number': 22, 'contract_name': 'dublin', 'name': 'TOWNSEND STREET', 'address': 'Townsend Street', 'position': {'lat': 53.345922, 'lng': -6.254614}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 12, 'available_bikes': 7, 'status': 'OPEN', 'la

2023-02-27 19:52:09,793 INFO sqlalchemy.engine.Engine [raw sql] (86, 10, 28, 1677527261000)
2023-02-27 19:52:09,812 INFO sqlalchemy.engine.Engine COMMIT
{'number': 35, 'contract_name': 'dublin', 'name': 'SMITHFIELD', 'address': 'Smithfield', 'position': {'lat': 53.347692, 'lng': -6.278214}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 8, 'available_bikes': 21, 'status': 'OPEN', 'last_update': 1677527422000}
2023-02-27 19:52:09,850 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
2023-02-27 19:52:09,851 INFO sqlalchemy.engine.Engine [raw sql] (35, 21, 8, 1677527422000)
2023-02-27 19:52:09,877 INFO sqlalchemy.engine.Engine COMMIT
{'number': 10, 'contract_name': 'dublin', 'name': 'DAME STREET', 'address': 'Dame Street', 'position': {'lat': 53.344007, 'lng': -6.266802}, 'banking': True, 'bonus': False, 'bike_stands': 16, 'available_bike_stands': 1, 'available_bikes': 15, 'status': 'OPEN', 'last_update': 1677526989000}
2023-02-27 19

2023-02-27 19:52:10,761 INFO sqlalchemy.engine.Engine [raw sql] (3, 6, 14, 1677527007000)
2023-02-27 19:52:10,781 INFO sqlalchemy.engine.Engine COMMIT
{'number': 40, 'contract_name': 'dublin', 'name': 'JERVIS STREET', 'address': 'Jervis Street', 'position': {'lat': 53.3483, 'lng': -6.266651}, 'banking': False, 'bonus': False, 'bike_stands': 21, 'available_bike_stands': 17, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1677527140000}
2023-02-27 19:52:10,820 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s)
2023-02-27 19:52:10,820 INFO sqlalchemy.engine.Engine [raw sql] (40, 3, 17, 1677527140000)
2023-02-27 19:52:10,842 INFO sqlalchemy.engine.Engine COMMIT
{'number': 29, 'contract_name': 'dublin', 'name': 'ORMOND QUAY UPPER', 'address': 'Ormond Quay Upper', 'position': {'lat': 53.346057, 'lng': -6.268001}, 'banking': False, 'bonus': False, 'bike_stands': 29, 'available_bike_stands': 14, 'available_bikes': 15, 'status': 'OPEN', 'last_update': 167752703800

In [15]:
import pandas as pd

df = pd.read_sql_table("station", engine)

2023-02-27 19:52:12,233 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-02-27 19:52:12,234 INFO sqlalchemy.engine.Engine [cached since 21.2s ago] ('dbbikes', 'station')
2023-02-27 19:52:12,313 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `station`
2023-02-27 19:52:12,314 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-27 19:52:12,352 INFO sqlalchemy.engine.Engine SELECT station.address, station.banking, station.bike_stand, station.bonus, station.contract_name, station.name, station.number, station.position_lat, station.position_lgt, station.status 
FROM station
2023-02-27 19:52:12,353 INFO sqlalchemy.engine.Engine [generated in 0.00153s] ()


In [16]:
display(df.head())

,address,banking,bike_stand,bonus,contract_name,name,number,position_lat,position_lgt,status
0,Smithfield North,0,None,0,dublin,SMITHFIELD NORTH,42,53.349562,NaN,OPEN
1,Parnell Square North,0,None,0,dublin,PARNELL SQUARE NORTH,30,53.353742,NaN,OPEN
2,Clonmel Street,0,None,0,dublin,CLONMEL STREET,54,53.336021,NaN,OPEN
3,Avondale Road,0,None,0,dublin,AVONDALE ROAD,108,53.359405,NaN,OPEN
4,James Street East,0,None,0,dublin,JAMES STREET EAST,20,53.336597,NaN,OPEN


In [17]:
sql = "select count(*) from station;"
print(engine.execute(sql).fetchall())

2023-02-27 19:52:12,427 INFO sqlalchemy.engine.Engine select count(*) from station;
2023-02-27 19:52:12,428 INFO sqlalchemy.engine.Engine [raw sql] ()
[(114,)]


In [18]:
sql = "select name from station limit 10;"
for row in engine.execute(sql):
    print(row)

2023-02-27 19:52:12,486 INFO sqlalchemy.engine.Engine select name from station limit 10;
2023-02-27 19:52:12,488 INFO sqlalchemy.engine.Engine [raw sql] ()
('SMITHFIELD NORTH',)
('PARNELL SQUARE NORTH',)
('CLONMEL STREET',)
('AVONDALE ROAD',)
('JAMES STREET EAST',)
('MOUNT STREET LOWER',)
('CHRISTCHURCH PLACE',)
('GRANTHAM STREET',)
('PEARSE STREET',)
('YORK STREET EAST',)


In [19]:
sql = """
select * from
(select available_bike_stands, available_bikes, number from availability 
 where available_bikes > 0 GROUP BY number) a1
 
 JOIN
 (select name, number from station) a2
ON a1.number = a2.number limit 10;
"""

for row in engine.execute(sql):
    print(row)

2023-02-27 19:52:12,525 INFO sqlalchemy.engine.Engine 
select * from
(select available_bike_stands, available_bikes, number from availability 
 where available_bikes > 0 GROUP BY number) a1
 
 JOIN
 (select name, number from station) a2
ON a1.number = a2.number limit 10;

2023-02-27 19:52:12,527 INFO sqlalchemy.engine.Engine [raw sql] ()
(0, 11, 30, 'PARNELL SQUARE NORTH', 30)
(15, 108, 20, 'JAMES STREET EAST', 20)
(24, 23, 6, 'CHRISTCHURCH PLACE', 6)
(12, 8, 18, 'GRANTHAM STREET', 18)
(1, 54, 32, 'PEARSE STREET', 32)
(18, 1, 13, 'FITZWILLIAM SQUARE WEST', 13)
(9, 113, 31, 'PARNELL STREET', 31)
(26, 48, 14, 'FOWNES STREET UPPER', 14)
(15, 10, 1, 'CLARENDON ROW', 1)
(17, 106, 23, 'CUSTOM HOUSE', 23)
